# Model Formulation

This notebook can be used to run initial model with DCOPF and a basic budget variable. Before running this nodebook please refer to `Test-System-Data-Processing.ipynb` to preprocess generator, lines, buses, and loads data. 

In [43]:
using JuMP, Gurobi
using DataFrames, CSV

include(joinpath("capex_model", "expansion_tools.jl"))
include(joinpath("capex_model", "model.jl"))

expansion (generic function with 1 method)

In [46]:
#run_model(load, basic_capex, save, "san_diego_system", "test_senario")
(buses, lines, gens, loads, variability, P) = load("san_diego_system")
model = expansion(buses, lines, gens, loads, variability, P)
save(model, "san_diego_system", "test_senario", buses, lines, gens, loads);

Set parameter Username
Set parameter LicenseID to value 2669913
Academic license - for non-commercial use only - expires 2026-05-22
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M2 Ultra
Thread count: 24 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 206976 rows, 140401 columns and 459039 nonzeros
Model fingerprint: 0x7265f0e4
Coefficient statistics:
  Matrix range     [1e-03, 1e+07]
  Objective range  [7e+00, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-03, 3e+03]
Presolve removed 140484 rows and 76380 columns
Presolve time: 0.18s
Presolved: 66492 rows, 64021 columns, 246045 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.55s

Barrier statistics:
 Dense cols : 152
 Free vars  : 138
 AA' NZ     : 3.619e+05
 Factor NZ  : 7.705e+06 (roughly 110 MB of memory)
 Factor Ops : 6.259e+09 (less than 1 second p

In [ ]:
#91s with contraints on new cap
#16s with no constraints on new cap
#7s with no constraints on cap or angles

In [ ]:
# TODOs
# - kmeans clustering & fix weighting
# - 
# - Ramping <=== seems important...
# - Get correct costs


## EXTRA
# Quardatic costs



## Questions for Qi ##
# a) costs? How do you handel the fact that costs are quadratic? Also why do they include c0?
# b) 1e-8 * 100 * (2700000 / 20 + 12500) # What are the units of this? and why do batteries cost less than solar??


# 3 mins to solve 1 week of cap ex